In [50]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from datetime import date

In [51]:
import warnings
warnings.filterwarnings("ignore")

In [52]:
df_train=pd.read_csv('train.csv')

In [53]:
df_train=df_train.dropna()

In [54]:
df_train.shape

(225493, 41)

In [55]:
df_train.shape

(225493, 41)

In [56]:
df_train.select_dtypes('object').columns

Index(['Date.of.Birth', 'Employment.Type', 'DisbursalDate',
       'PERFORM_CNS.SCORE.DESCRIPTION', 'AVERAGE.ACCT.AGE',
       'CREDIT.HISTORY.LENGTH'],
      dtype='object')

In [57]:
def cns_desc(x):
    if x<300:
        return 0
    elif (x>=300) and (x<=350):
        return 1
    elif (x>350) and (x<=570):
        return 2
    elif (x>570) and (x<=630):
        return 3
    elif (x>630) and (x<=705):
        return 4
    else:
        return 5

In [58]:
def fix_date(x):

    if x.year >= 2049:
        year = x.year - 100
    else:
        year = x.year
    return datetime.date(year,x.month,x.day)
df_train["Date.of.Birth"]=pd.to_datetime(df_train["Date.of.Birth"])
df_train["Date.of.Birth"] = df_train["Date.of.Birth"].apply(fix_date)



In [59]:
df_train["Date.of.Birth"]=pd.to_datetime(df_train["Date.of.Birth"])

df_train["DisbursalDate"]=pd.to_datetime(df_train["DisbursalDate"])
df_train["DisbursalDate_Month"]=df_train["DisbursalDate"].dt.month

df_train["Age_at_disbursement"]=((df_train['DisbursalDate']-df_train["Date.of.Birth"])/365).apply(lambda x:x.days)

df_train["Employment.Type"]=df_train["Employment.Type"].map({'Salaried':0,"Self employed":1,np.nan:2})

df_train["PERFORM_CNS.SCORE.DESCRIPTION"]=df_train["PERFORM_CNS.SCORE"].apply(cns_desc)

df_train["AVERAGE.ACCT.AGE"]=df_train["AVERAGE.ACCT.AGE"].apply(lambda x:(int((x.split("y"))[0])*12)+(int((x.split())[1].split("m")[0])))

df_train["CREDIT.HISTORY.LENGTH"]=df_train["CREDIT.HISTORY.LENGTH"].apply(lambda x:(int((x.split("y"))[0])*12)+(int((x.split())[1].split("m")[0])))

In [60]:
df_train['NO.OF.ACCTS']=df_train['PRI.NO.OF.ACCTS']+df_train['SEC.NO.OF.ACCTS']
df_train['ACTIVE.ACCTS']=df_train['PRI.ACTIVE.ACCTS']+df_train['SEC.ACTIVE.ACCTS']
df_train['OVERDUE.ACCTS']=df_train['PRI.OVERDUE.ACCTS']+df_train['SEC.OVERDUE.ACCTS']
df_train['CURRENT.BALANCE']=df_train['PRI.CURRENT.BALANCE']+df_train['SEC.CURRENT.BALANCE']
df_train['SANCTIONED.AMOUNT']=df_train['PRI.SANCTIONED.AMOUNT']+df_train['SEC.SANCTIONED.AMOUNT']
df_train['DISBURSED.AMOUNT']=df_train['PRI.DISBURSED.AMOUNT']+df_train['SEC.DISBURSED.AMOUNT']
df_train['INSTAL.AMT']=df_train['PRIMARY.INSTAL.AMT']+df_train['SEC.INSTAL.AMT']

df_train=df_train.drop(['PRI.NO.OF.ACCTS','SEC.NO.OF.ACCTS','PRI.ACTIVE.ACCTS','SEC.ACTIVE.ACCTS','PRI.OVERDUE.ACCTS','SEC.OVERDUE.ACCTS','PRI.CURRENT.BALANCE','SEC.CURRENT.BALANCE','PRI.SANCTIONED.AMOUNT','SEC.SANCTIONED.AMOUNT','PRI.DISBURSED.AMOUNT','SEC.DISBURSED.AMOUNT','PRIMARY.INSTAL.AMT','SEC.INSTAL.AMT'],axis=1)

In [61]:
df_train=df_train.drop(columns=['Date.of.Birth',"DisbursalDate"])

In [62]:
df_train[df_train['DISBURSED.AMOUNT']==df_train['DISBURSED.AMOUNT'].max()]

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Employment.Type,State_ID,...,loan_default,DisbursalDate_Month,Age_at_disbursement,NO.OF.ACCTS,ACTIVE.ACCTS,OVERDUE.ACCTS,CURRENT.BALANCE,SANCTIONED.AMOUNT,DISBURSED.AMOUNT,INSTAL.AMT
71593,598208,47773,85964,59.33,20,16700,86,6434,1,5,...,1,10,50,2,1,0,36041,1000000000,1000000000,0


In [63]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 225493 entries, 0 to 233153
Data columns (total 34 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   UniqueID                             225493 non-null  int64  
 1   disbursed_amount                     225493 non-null  int64  
 2   asset_cost                           225493 non-null  int64  
 3   ltv                                  225493 non-null  float64
 4   branch_id                            225493 non-null  int64  
 5   supplier_id                          225493 non-null  int64  
 6   manufacturer_id                      225493 non-null  int64  
 7   Current_pincode_ID                   225493 non-null  int64  
 8   Employment.Type                      225493 non-null  int64  
 9   State_ID                             225493 non-null  int64  
 10  Employee_code_ID                     225493 non-null  int64  
 11  MobileNo_Avl_

### Import statements for models

In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [67]:
import lightgbm as lgb

In [21]:
X=df_train.drop(['loan_default'],axis=1)
y=df_train['loan_default']

In [22]:
from sklearn.model_selection import KFold,cross_val_score,train_test_split

In [42]:
d={}
lr=LogisticRegression()
kfolds=KFold(n_splits=5,shuffle=True,random_state=1)
scorers=cross_val_score(lr,X,y,scoring='roc_auc')
print(np.mean(scorers))
print(np.std(scorers))
d['logistic_regression']=scorers

0.5543033112858818
0.007687559667951311


In [45]:
d={}
dt=DecisionTreeClassifier()
kfolds=KFold(n_splits=5,shuffle=True,random_state=1)
scorers=cross_val_score(dt,X,y,scoring='roc_auc')
print(np.mean(scorers))
print(np.std(scorers))
d['Decision_Tree']=scorers

0.5179952600924311
0.007535133174971846


In [47]:
d={}
rfc=RandomForestClassifier()
kfolds=KFold(n_splits=5,shuffle=True,random_state=1)
scorers=cross_val_score(rfc,X,y,scoring='roc_auc')
print(np.mean(scorers))
print(np.std(scorers))
d['Random_forest']=scorers

0.6089171705159476
0.01389497825310791


In [48]:
d

{'Random_forest': array([0.63264253, 0.60395348, 0.59225692, 0.61486823, 0.6008647 ])}

In [69]:
d={}
lgbc=lgb.LGBMClassifier()
kfolds=KFold(n_splits=5,shuffle=True,random_state=1)
scorers=cross_val_score(lgbc,X,y,scoring='roc_auc')
print(np.mean(scorers))
print(np.std(scorers))
d['Random_forest']=scorers

0.6210544223495829
0.02430213370190127
